In [1]:
import parent

In [2]:
import data

In [3]:
d1 = data.get_dataset_triangles("test")

In [4]:
import inverseConsistentNet
import networks

In [6]:
import parent
import data
import networks
import visualize
import train
import inverseConsistentNet
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import os



d1_triangles, d2_triangles = data.get_dataset_triangles(
    "train", data_size=50, hollow=True
)
d1_triangles_test, d2_triangles_test = data.get_dataset_triangles(
    "test", data_size=50, hollow=True
)

network = networks.tallUNet2

d1, d2, d1_t, d2_t = (d1_triangles, d2_triangles, d1_triangles_test, d2_triangles_test)

net = inverseConsistentNet.InverseConsistentNet(
    network(), 2048, next(iter(d1))[0].size()
)
net.cuda()

InverseConsistentNet(
  (regis_net): UNet2(
    (downConvs): ModuleList(
      (0): Conv2d(2, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): Conv2d(64, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (upConvs): ModuleList(
      (0): ConvTranspose2d(48, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): ConvTranspose2d(96, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (2): ConvTranspose2d(192, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): ConvTranspose2d(512, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    )
    (batchNorms): ModuleList(
      (0): BatchNorm2d(16, eps=1e-05

In [30]:
net.load_state_dict(torch.load("../results/batchnormtriangles/network.trch"))

<All keys matched successfully>

In [31]:
image_A, image_B = (x[0].cuda() for x in next(zip(d1_t, d2_t)))

In [41]:
visualize.visualizeRegistration(net, image_B, image_A, 3, "../results/tmp.png")

Diffeomorphism Failures per batch
tensor(13)


<Figure size 2400x1600 with 0 Axes>

In [34]:
net(image_A, image_B)

[tensor(0.0384, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0249, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.0136, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(13.2590, device='cuda:0', grad_fn=<MulBackward0>)]

In [36]:
net.phi_AB.shape

torch.Size([128, 2, 50, 50])

In [38]:
phis = []
for _ in range(60):
    image_A, image_B = (x[0].cuda() for x in next(zip(d1_t, d2_t)))
    net(image_A, image_B)
    phi = net.phi_AB.cpu().detach()
    phis.append(phi)

In [39]:
phis_array = np.concatenate(phis, 0)

In [40]:
phis_array.shape

(7680, 2, 50, 50)

In [42]:
import pickle
with open("phi_map_array.pickle", "wb") as f:
    pickle.dump(phis_array, f)

NameError: name 'scp' is not defined